In [2]:
import json
from tqdm import tqdm
import os
import json
#import pickle
#from eval_retrieval_results import RunFuser

In [3]:
eval_year = '2020'

# Load qrel and topics data

give you some statistics about the data you use

In [4]:
if eval_year == "2019":
    # global var for 2019
    topic_dir = './CAsT/topics/2019_evaluation_topics_v1.0.json'
    qrel_file_dir = './CAsT/qrels/2019qrels.txt'
    raw_query_field = 'raw_utterance'
elif eval_year == "2020":
    # global vars for 2020
    topic_dir = "./CAsT/topics//2020_manual_evaluation_topics_v1.0.json"
    qrel_file_dir = "./CAsT/qrels/2020qrels.txt"
    raw_query_field = 'raw_utterance'
elif eval_year == "2021":
    topic_dir = './CAsT/topics/2021_manual_evaluation_topics_v1.0.json'
    qrel_file_dir = './CAsT/qrels/2021qrels.txt'
    raw_query_field = 'raw_utterance'
elif eval_year == "2022":
    topic_dir = "./CAsT/topics/2022_evaluation_topics_flattened_duplicated_v1.0json"
    qrel_file_dir = './CAsT/topics/2022qrels.txt'
    raw_query_field = 'utterance'
else:
    raise ValueError("eval_year must be one of 2019, 2020, 2021, 2022")

In [5]:
# create all qid list
all_qids = []
with open(topic_dir, 'r') as topics:
    topics = json.load(topics)
for session in topics:
    session_num = str(session["number"])
    for turn_id, conversations in enumerate(session["turn"]):
        #query = conversations[raw_query_field]
        conversation_num = str(conversations["number"])
        qid = session_num + "_" + conversation_num
        if qid in all_qids:
            pass
        else:
            all_qids.append(qid)

all_qids = list(set(all_qids))

print(f"totally {len(all_qids)} number of qids to evaluate")

totally 216 number of qids to evaluate


# Evaluate

## Define Evaluate Functions

The reason 2021 is different from other CAsT years is because other years are evaluated on passage, but CAsT-2021 is evaluated on document. See TREC CAsT 2021 Overview paper for details.

In [6]:

    
def eval2021(hits_num, run_file_dir:str, run_name:str, qrel_file_dir:str)-> None:
    '''
    Only 2021 need to docize the runfile
    '''
    docize_file_dir = "/data_hdd/dayu/TREC/run_results/official_docize.py"
    
    
    print("evaluating...")
    os.system(f'python3 {docize_file_dir} --run_file_path {run_file_dir} --run_name {run_name}')
    
    # os.system(f'python -m pyserini.eval.trec_eval -c -M 1000 -m map --level_for_rel 2 {qrel_file_dir} /data_hdd/dayu/TREC/run_results/{run_name}-converted.run')
    # os.system(f'python -m pyserini.eval.trec_eval -c -M 1000 -m mrr --level_for_rel 2 {qrel_file_dir} /data_hdd/dayu/TREC/run_results/{run_name}-converted.run')
    # os.system(f'python -m pyserini.eval.trec_eval -c -M 1000 -m recall --level_for_rel 2 {qrel_file_dir} /data_hdd/dayu/TREC/run_results/{run_name}-converted.run')
    # os.system(f'python -m pyserini.eval.trec_eval -c -M {hits_num} -m recall.{hits_num} --level_for_rel 2 {qrel_file_dir} /data_hdd/dayu/TREC/run_results/{run_name}-converted.run')
    # os.system(f'python -m pyserini.eval.trec_eval -c -M 1000 -m ndcg_cut.3 {qrel_file_dir} /data_hdd/dayu/TREC/run_results/{run_name}-converted.run')
    # os.system(f'python -m pyserini.eval.trec_eval -c -q -m all_trec {qrel_file_dir} /data_hdd/dayu/TREC/run_results/{run_name}-converted.run')

    #os.system(f'python -m pyserini.eval.trec_eval -c -m ndcg_cut.3 {qrel_file_dir} {run_file_dir}')
    os.system(f'python -m pyserini.eval.trec_eval -c -m ndcg_cut.5 {qrel_file_dir} {run_file_dir}')
    os.system(f'python -m pyserini.eval.trec_eval -c -m P.5 {qrel_file_dir} {run_file_dir}')
    os.system(f'python -m pyserini.eval.trec_eval -c -m recall.100 {qrel_file_dir} {run_file_dir}')
    os.system(f'python -m pyserini.eval.trec_eval -c -m map {qrel_file_dir} {run_file_dir}')
    
    
    print("Evaluation Done")
    
    
def eval(hits_num, run_file_dir:str, qrel_file_dir:str)-> None:
    '''
    
    '''
    #docize_file_dir = "/data_hdd/dayu/TREC/run_results/official_docize.py"
    
    
    print("evaluating...")
    #os.system(f'python3 {docize_file_dir} --run_file_path {run_file_dir} --run_name {run_name}')
    
    #os.system(f'python -m pyserini.eval.trec_eval -c -m map_cut.1000 --level_for_rel 2 {qrel_file_dir} {run_file_dir}')
    #os.system(f'python -m pyserini.eval.trec_eval -c -m ndcg_cut.3 {qrel_file_dir} {run_file_dir}')
    os.system(f'python -m pyserini.eval.trec_eval -c -m ndcg_cut.5 {qrel_file_dir} {run_file_dir}')
    os.system(f'python -m pyserini.eval.trec_eval -c -m P.5 {qrel_file_dir} {run_file_dir}')
    os.system(f'python -m pyserini.eval.trec_eval -c -m recall.100 {qrel_file_dir} {run_file_dir}')
    os.system(f'python -m pyserini.eval.trec_eval -c -m map {qrel_file_dir} {run_file_dir}')
    #os.system(f'python -m pyserini.eval.trec_eval -c -q -m all_trec {qrel_file_dir} {run_file_dir}')
    
    print("Evaluation Done")

    

# Reproduce the paper results

In [7]:
# EVAL CAST-19
run_file_dir = 'sample/ranking_results/2019-dedup.run'
qrel_dir = 'CAsT/qrels/2019qrels.txt'
eval(1000, run_file_dir,qrel_dir)

evaluating...


2023-07-12 17:23:53.601067: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.5', 'CAsT/qrels/2019qrels.txt', 'sample/ranking_results/2019-dedup.run']
Results:
ndcg_cut_5            	all	0.3821



2023-07-12 17:23:58.030683: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'P.5', 'CAsT/qrels/2019qrels.txt', 'sample/ranking_results/2019-dedup.run']
Results:
P_5                   	all	0.4751



2023-07-12 17:24:02.522057: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'recall.100', 'CAsT/qrels/2019qrels.txt', 'sample/ranking_results/2019-dedup.run']
Results:
recall_100            	all	0.3575



2023-07-12 17:24:07.106836: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'map', 'CAsT/qrels/2019qrels.txt', 'sample/ranking_results/2019-dedup.run']
Results:
map                   	all	0.2095

Evaluation Done


In [8]:
# EVAL CAST-20
run_file_dir = 'sample/ranking_results/2020.run'
qrel_dir = 'CAsT/qrels/2020qrels.txt'
eval(1000, run_file_dir,qrel_dir)

evaluating...


2023-07-12 17:24:32.233194: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.5', 'CAsT/qrels/2020qrels.txt', 'sample/ranking_results/2020.run']
Results:
ndcg_cut_5            	all	0.2281



2023-07-12 17:24:37.000418: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'P.5', 'CAsT/qrels/2020qrels.txt', 'sample/ranking_results/2020.run']
Results:
P_5                   	all	0.2981



2023-07-12 17:24:41.796850: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'recall.100', 'CAsT/qrels/2020qrels.txt', 'sample/ranking_results/2020.run']
Results:
recall_100            	all	0.3116



2023-07-12 17:24:46.423015: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'map', 'CAsT/qrels/2020qrels.txt', 'sample/ranking_results/2020.run']
Results:
map                   	all	0.1462

Evaluation Done


In [9]:
# EVAL CAST-21
run_file_dir = 'sample/ranking_results/2021-docize.run'
qrel_dir = 'CAsT/qrels/2021qrels.txt'
eval(1000, run_file_dir,qrel_dir)

evaluating...


2023-07-12 17:24:51.037183: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.5', 'CAsT/qrels/2021qrels.txt', 'sample/ranking_results/2021-docize.run']
Results:
ndcg_cut_5            	all	0.2712



2023-07-12 17:24:55.613792: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'P.5', 'CAsT/qrels/2021qrels.txt', 'sample/ranking_results/2021-docize.run']
Results:
P_5                   	all	0.3316



2023-07-12 17:25:00.091820: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'recall.100', 'CAsT/qrels/2021qrels.txt', 'sample/ranking_results/2021-docize.run']
Results:
recall_100            	all	0.3453



2023-07-12 17:25:04.616520: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'map', 'CAsT/qrels/2021qrels.txt', 'sample/ranking_results/2021-docize.run']
Results:
map                   	all	0.1720

Evaluation Done


In [10]:
# EVAL CAST-22
run_file_dir = 'sample/ranking_results/2022.run'
qrel_dir = 'CAsT/qrels/2022qrels.txt'
eval(1000, run_file_dir,qrel_dir)

evaluating...


2023-07-12 17:25:09.119292: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.5', 'CAsT/qrels/2022qrels.txt', 'sample/ranking_results/2022.run']
Results:
ndcg_cut_5            	all	0.1998



2023-07-12 17:25:13.643599: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'P.5', 'CAsT/qrels/2022qrels.txt', 'sample/ranking_results/2022.run']
Results:
P_5                   	all	0.2687



2023-07-12 17:25:18.215659: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'recall.100', 'CAsT/qrels/2022qrels.txt', 'sample/ranking_results/2022.run']
Results:
recall_100            	all	0.1527



2023-07-12 17:25:22.775952: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/dayu/anaconda3/envs/chatty_goose/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/usa/dayu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'map', 'CAsT/qrels/2022qrels.txt', 'sample/ranking_results/2022.run']
Results:
map                   	all	0.0956

Evaluation Done
